#### Stdlib dataclasses and pydantic dataclasses

##### Convert stdlib dataclasses into pydantic dataclasses

Stdlib dataclasses (nested or not) can be easily converted into pydantic dataclasses by just decorating them with `pydantic.dataclasses.dataclass`. Pydantic will enhance the given stdlib dataclass but won't alter the default behaviour (i.e. without validation). It will instead create a wrapper around it to trigger validation that will act like a plain proxy. The stdlib dataclass can still be accessed via the `__dataclass__` attribute.

In [1]:
import dataclasses
from datetime import datetime
from typing import Optional

import pydantic
from pydantic import BaseModel, ValidationError
from pydantic.dataclasses import dataclass as pydantic_dataclass, set_validation

In [2]:
@dataclasses.dataclass
class Meta:
    modified_date: Optional[datetime]
    seen_count: int

In [3]:
@dataclasses.dataclass
class File(Meta):
    filename: str

In [4]:
# `ValidatedFile` will be a proxy around `File`
ValidatedFile = pydantic.dataclasses.dataclass(File)

# the original dataclass is the `__dataclass__` attribute
assert ValidatedFile.__dataclass__ is File

In [5]:
validated_file = ValidatedFile(
    filename=b"thefilename",
    modified_date="2020-01-01T00:00",
    seen_count="7",
)
print(f"{validated_file = }")

validated_file = File(modified_date=datetime.datetime(2020, 1, 1, 0, 0), seen_count=7, filename='thefilename')


In [6]:
try:
    item = ValidatedFile(
        filename=["not", "a", "string"],
        modified_date=None,
        seen_count=3,
    )
except pydantic.ValidationError as e:
    print(e)

1 validation error for File
filename
  str type expected (type=type_error.str)


In [7]:
# `File` is not altered and still does no validation by default
print(f"{File(filename=['not', 'a', 'string'], modified_date=None, seen_count=3) = }")

File(filename=['not', 'a', 'string'], modified_date=None, seen_count=3) = File(modified_date=None, seen_count=3, filename=['not', 'a', 'string'])


##### Choose when to trigger validation

As soon as your stdlib dataclass has been decorated with pydantic dataclass decorator, magic methods have been added to validate input data. If you want, you can still keep using your dataclass and choose when to trigger it.

In [8]:
@dataclasses.dataclass
class User:
    id: int
    name: str

In [9]:
# Enhance stdlib dataclass
pydantic_dataclass(User)

In [10]:
user_1 = User(id="whatever", name="I want")

In [11]:
# validate data of `user_1`
try:
    validate = user_1.__pydantic_validate_values__()
except ValidationError as e:
    print(e)

1 validation error for User
id
  value is not a valid integer (type=type_error.integer)


In [12]:
try:
    with set_validation(User, True):
        item = User(id="whatever", name="I want")
except ValidationError as e:
    print(e)

1 validation error for User
id
  value is not a valid integer (type=type_error.integer)


##### Inherit from stdlib dataclasses

Stdlib dataclasses (nested or not) can also be inherited and pydantic will automatically validate all the inherited fields.

In [13]:
@dataclasses.dataclass
class Z:
    z: int

In [14]:
@dataclasses.dataclass
class Y(Z):
    y: int = 0

In [15]:
@pydantic.dataclasses.dataclass
class X(Y):
    x: int = 0

In [16]:
foo = X(x=b"1", y="2", z="3")
print(f"{foo = }")

foo = X(z=3, y=2, x=1)


In [17]:
try:
    item = X(z="pika")
except pydantic.ValidationError as e:
    print(e)

1 validation error for X
z
  value is not a valid integer (type=type_error.integer)


##### Use of stdlib dataclasses with `BaseModel`

Bear in mind that stdlib dataclasses (nested or not) are __automatically converted__ into pydantic dataclasses when mixed with `BaseModel`! Furthermore the generated pydantic dataclass will have the __exact same configuration__ (`order`, `frozen`, ...) as the original one.

In [18]:
@dataclasses.dataclass(frozen=True)
class FrozenUser:
    name: str

In [19]:
@dataclasses.dataclass
class CustomFile:
    filename: str
    last_modification_time: Optional[datetime] = None

In [20]:
class Foo(BaseModel):
    file: CustomFile
    user: Optional[FrozenUser] = None

In [21]:
file = CustomFile(
    filename=["not", "a", "string"],
    last_modification_time="2020-01-01T00:00",
)  # nothing is validated as expected
print(f"{file = }")

file = CustomFile(filename=['not', 'a', 'string'], last_modification_time='2020-01-01T00:00')


In [22]:
try:
    item = Foo(file=file)
except ValidationError as e:
    print(e)

1 validation error for Foo
file -> filename
  str type expected (type=type_error.str)


In [23]:
foo = Foo(file=CustomFile(filename="myfile"), user=FrozenUser(name="pika"))
try:
    foo.user.name = "bulbi"
except dataclasses.FrozenInstanceError as e:
    print(e)

cannot assign to field 'name'


##### Use custom types

Since stdlib dataclasses are automatically converted to add validation using custom types may cause some unexpected behaviour. In this case you can simply add `arbitrary_types_allowed` in the config!

In [24]:
class ArbitraryType:
    def __init__(self, value):
        self.value = value

    def __repr__(self):
        return f"ArbitraryType(value={self.value!r})"

In [25]:
@dataclasses.dataclass
class DC:
    a: ArbitraryType
    b: str

In [26]:
# valid as it is a builtin dataclass without validation
my_dc = DC(a=ArbitraryType(value=3), b="qwe")

In [27]:
try:
    class Model(pydantic.BaseModel):
        dc: DC
        other: str

    Model(dc=my_dc, other="other")
except RuntimeError as e:  # invalid as it is now a pydantic dataclass
    print(e)

no validator found for <class '__main__.ArbitraryType'>, see `arbitrary_types_allowed` in Config


In [28]:
class Model(pydantic.BaseModel):
    dc: DC
    other: str

    class Config:
        arbitrary_types_allowed = True

In [29]:
m = Model(dc=my_dc, other="other")
print(repr(m))

Model(dc=DC(a=ArbitraryType(value=3), b='qwe'), other='other')
